In [1]:
from utility import db_connect, query2csv
from settings import  DBNAME, DBPASS, DBUSER, DBHOST

In [2]:
qsql="""
select 
  ar.analysis_area_name as city,
  aa.mode,
  array_agg(analysis_area_id order by analysis_area_id) as analysis_area_id_list
from
baa.analysis_area_regions as ar,
baa.analysis_areas as aa
where ar.analysis_area_regions_id = aa.analysis_area_regions_id
group by 1,2
"""
csvfile='city_mode_group.csv'
query2csv(qsql,csvfile)

### Create monthly city mode factor group table

```sql
CREATE TABLE baa_ex_sus.fg_city_mode_monthly
(
  city character varying(50) NOT NULL,
  mode baa.bp_mode NOT NULL,
  analysis_area_id_list integer[]
)
```
### Populate monthly city mode factor group table

```sql
insert into baa_ex_sus.fg_city_mode_monthly(
with city_mode_group as (
select 
  ar.analysis_area_name as city,
  aa.mode,
  array_agg(analysis_area_id order by analysis_area_id) as analysis_area_id_list
from
baa.analysis_area_regions as ar,
baa.analysis_areas as aa
where ar.analysis_area_regions_id = aa.analysis_area_regions_id
group by 1,2
)
```

In [3]:
qsql="""
select * from baa_ex_sus.fg_city_mode_monthly
"""
csvfile='fg_city_mode_monthly.csv'
query2csv(qsql,csvfile)

In [4]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
monthly_madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
)
select analysis_area_id, year, month, madt from monthly_madt_exclude_holiday
order by 1,2,3
"""
csvfile='monthly_madt_exclude_holiday_ex_sus.csv'
query2csv(qsql,csvfile)

In [5]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
monthly_madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as fm
from
  monthly_madt_exclude_holiday as madt_nh inner join aadt using(analysis_area_id, year)
where
  aadt.aadt <> 0  
)
select * from fm
order by 1,3,2
"""
csvfile='monthly_factor_ex_sus.csv'
query2csv(qsql,csvfile)

In [6]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
monthly_madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as fm
from
  monthly_madt_exclude_holiday as madt_nh inner join aadt using(analysis_area_id, year)
where
  aadt.aadt <> 0  
),
city_mode_group as (
select 
  ar.analysis_area_name as city,
  aa.mode,
  array_agg(analysis_area_id order by analysis_area_id) as analysis_area_id_list
from
  baa.analysis_area_regions as ar,
  baa.analysis_areas as aa
where ar.analysis_area_regions_id = aa.analysis_area_regions_id
  group by 1,2  
)  
select
  cmg.city,
  cmg.mode,
  cmg.analysis_area_id_list,
  fm.month,
  fm.year,
  round(avg(fm.fm), 2) as fm_cmp
 
from
fm inner join city_mode_group as cmg
on fm.analysis_area_id = Any(cmg.analysis_area_id_list::int[])
group by 1,2,3,5,4
order by 1,2,3,5,4
"""
csvfile='monthly_factor_group_ex_sus.csv'
query2csv(qsql,csvfile)